# From a REST API to a Dataset

In [1]:
# Import Libraries

import requests
import base64
import json
import pandas as pd
from pandasql import sqldf

# Set Credentials

clientID = ""
clientSecret = ""
playlist_id = '6SA68dT9UJFEbUVORRUUOJ'


def get_session_token(clientID, clientSecret):
    # // Makes an API call using HTTP POST method to generate a Token
    url = "https://accounts.spotify.com/api/token"
    headers = {}
    data = {}
    raw_cred = f"{clientID}:{clientSecret}"
    encoded_cred = raw_cred.encode('ascii')
    base64_cred = base64.b64encode(encoded_cred)
    auth_message = base64_cred.decode('ascii')

    headers['Authorization'] = f"Basic {auth_message}"
    data['grant_type'] = "client_credentials"

    r = requests.post(url, headers=headers, data=data)

    token = r.json()['access_token']
    return(token)
    
def get_playlist_items(token, playlist_id):
    # // Makes an API Call using HTTP GET method and the Token previously generated to fetch all 
    #    items from the playlist
    
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks?fields=items(track(name, album.name, artists.name, id))"
    payload={}
    headers = {
     "Authorization": "Bearer " + str(token)
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    # Cleansing and Transformation
    playlist_df = pd.json_normalize(response.json().get('items'))
    # Rename columns for readability
    playlist_df.rename(columns={"track.album.name": "album", "track.artists": "artist", "track.id":"id", "track.name":"name" }, inplace = True)
    # Rearrange columns for readability    
    playlist_df = playlist_df[['id','name','artist','album']]
    # Clean the artist column
    for i in range (0,len(playlist_df['artist'])):
        playlist_df['artist'][i] = playlist_df['artist'][i][0].get('name')
    return (playlist_df)

def get_track_list(playlist_df):
    # // From the playlist dataframe, extract all track IDs.
    
    track_ids = tuple(list(playlist_df['id']))
    track_ids = ','.join(track_ids)
    track_ids = track_ids.replace("'", "")
    return (track_ids)


def get_audio_features(token, track_ids):
    # // Makes an API Call using HTTP GET method, the Auth Token and the Track ID list to fetch all tracks' audio 
    #    features
    url = f"https://api.spotify.com/v1/audio-features?ids={track_ids}"
    payload={}
    headers = {
     "Authorization": "Bearer " + str(token)
    }
    response = requests.request("GET", url, headers=headers, data=payload)
    audio_features = pd.json_normalize(response.json().get('audio_features'))
    return (audio_features)

def join_dataframes (playlist_df, audio_features):
    # Merge both dataframes by Key ID
    
    playlist_features = pd.merge(playlist_df,
                                 audio_features,
                                 how="inner",
                                 left_on="id",
                                 right_on="id")
    return (playlist_features)

    

In [2]:
# Functions Execute
# Generate Token
token = get_session_token(clientID, clientSecret)
# Generate Playlist Items DF
playlist_df = get_playlist_items(token, playlist_id)
# Extract Tracks List
tracks = get_track_list(playlist_df)
# Extract Audio features
audio_features = get_audio_features(token, tracks)
# Merge to create final Dataframe
playlist_audio_features = join_dataframes(playlist_df, audio_features)


In [3]:
playlist_audio_features.head()

,id,name,artist,album,danceability,energy,key,loudness,mode,speechiness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,7lQWRAjyhTpCWFC0jmclT4,Gangsta's Paradise,Coolio,Gangsta's Paradise,0.647,0.514,8,-10.050,1,0.0593,...,0.000000,0.3980,0.387,79.974,audio_features,spotify:track:7lQWRAjyhTpCWFC0jmclT4,https://api.spotify.com/v1/tracks/7lQWRAjyhTpC...,https://api.spotify.com/v1/audio-analysis/7lQW...,240693,4
1,2UY4BdOZ7Ncfke96BBUTzM,Erase / Rewind,The Cardigans,Gran Turismo,0.732,0.681,11,-7.103,0,0.0317,...,0.001120,0.1100,0.699,103.720,audio_features,spotify:track:2UY4BdOZ7Ncfke96BBUTzM,https://api.spotify.com/v1/tracks/2UY4BdOZ7Ncf...,https://api.spotify.com/v1/audio-analysis/2UY4...,215760,4
2,3OmZjm0HACAylDWEIraO7i,Queen of Apology,The Sounds,Dying to Say This to You (European Version),0.540,0.880,1,-2.798,0,0.0454,...,0.000027,0.0428,0.583,152.146,audio_features,spotify:track:3OmZjm0HACAylDWEIraO7i,https://api.spotify.com/v1/tracks/3OmZjm0HACAy...,https://api.spotify.com/v1/audio-analysis/3OmZ...,186466,4
3,20CNpCKq1oTdvekXaboyeq,Friday I'm In Love,The Cure,Wish,0.526,0.763,0,-12.663,0,0.0341,...,0.000068,0.3590,0.514,136.205,audio_features,spotify:track:20CNpCKq1oTdvekXaboyeq,https://api.spotify.com/v1/tracks/20CNpCKq1oTd...,https://api.spotify.com/v1/audio-analysis/20CN...,215160,4
4,0M1CyBWblWSkeh0UdYH8pn,Happy House,Siouxsie and the Banshees,Kaleidoscope (Remastered & Expanded),0.543,0.705,7,-8.062,1,0.0465,...,0.188000,0.2370,0.334,119.252,audio_features,spotify:track:0M1CyBWblWSkeh0UdYH8pn,https://api.spotify.com/v1/tracks/0M1CyBWblWSk...,https://api.spotify.com/v1/audio-analysis/0M1C...,231707,4


# Analyzing results

In [4]:
pysqldf = lambda q: sqldf(q, globals())

## Most "Danceable"

In [5]:
query = "SELECT name, artist, album, danceability FROM playlist_audio_features ORDER BY danceability DESC LIMIT 5;"
pysqldf(query)

,name,artist,album,danceability
0,In Da Club,50 Cent,Get Rich Or Die Tryin',0.899
1,Cloud 9,Jamiroquai,Automaton,0.885
2,Every Morning,Sugar Ray,14:59,0.833
3,Sneakin' Out The Hospital - Remastered 2009,Beastie Boys,Hello Nasty (Deluxe Version/Remastered 2009),0.833
4,Song For Junior,Beastie Boys,Hello Nasty,0.814


## Happiest Song

In [6]:
query = "SELECT name, artist, album, valence FROM playlist_audio_features ORDER BY valence DESC LIMIT 5"
pysqldf(query)

,name,artist,album,valence
0,Every Morning,Sugar Ray,14:59,0.975
1,1979 - Remastered 2012,The Smashing Pumpkins,Mellon Collie And The Infinite Sadness (Deluxe...,0.964
2,Abracadabra,Steve Miller Band,Abracadabra,0.963
3,You Spin Me Round (Like a Record),Dead Or Alive,Youthquake,0.960
4,Starchild,Jamiroquai,Dynamite,0.960


## Saddest songs

In [7]:
query = "SELECT name, artist, album, valence FROM playlist_audio_features ORDER BY valence ASC LIMIT 5"
pysqldf(query)

,name,artist,album,valence
0,Why Does My Heart Feel so Bad?,Moby,Play,0.0393
1,Afterglow,Wilkinson,Lazers Not Included,0.0398
2,Black Hole Sun,Soundgarden,Superunknown (Deluxe Edition),0.1470
3,Wish,Nine Inch Nails,Broken,0.1490
4,Pure Morning,Placebo,Without You I'm Nothing,0.2020


## Most Energetic Songs

In [8]:
query = "SELECT name, artist, album, energy FROM playlist_audio_features ORDER BY energy DESC LIMIT 5"
pysqldf(query)

,name,artist,album,energy
0,Wish,Nine Inch Nails,Broken,0.989
1,Simple Song,Avail,4AM Friday,0.984
2,Block Rockin' Beats,The Chemical Brothers,Block Rockin’ Beats,0.983
3,I Predict A Riot,Kaiser Chiefs,Employment,0.979
4,Get Free,The Vines,Highly Evolved,0.978
